In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [212]:
train = pd.read_csv('/users/jjong/desktop/vscode/Dacon_src/jeju/train.csv')
test = pd.read_csv('/users/jjong/desktop/vscode/Dacon_src/jeju/test.csv')
trade = pd.read_csv('/users/jjong/desktop/vscode/Dacon_src/jeju/international_trade.csv')
sample = pd.read_csv('/users/jjong/desktop/vscode/Dacon_src/jeju/sample_submission.csv')

In [213]:
#시계열 특성을 학습에 반영하기 위해 timestamp를 월, 일, 시간으로 나눕니다
train['year'] = train['timestamp'].apply(lambda x : int(x[0:4]))
train['month'] = train['timestamp'].apply(lambda x : int(x[5:7]))
train['day'] = train['timestamp'].apply(lambda x : int(x[8:10]))

test['year'] = test['timestamp'].apply(lambda x : int(x[0:4]))
test['month'] = test['timestamp'].apply(lambda x : int(x[5:7]))
test['day'] = test['timestamp'].apply(lambda x : int(x[8:10]))

trade['year'] = trade['기간'].apply(lambda x : int(x[0:4]))
trade['month'] = trade['기간'].apply(lambda x : int(x[5:7]))

In [214]:
def english(data, col):
    data[col] = data[col].apply(lambda x: 'TG' if '감귤' in x else x)
    data[col] = data[col].apply(lambda x: 'BC' if '브로콜리' in x else x)
    data[col] = data[col].apply(lambda x: 'RD' if '무' in x else x)
    data[col] = data[col].apply(lambda x: 'CR' if '당근' in x else x)
    data[col] = data[col].apply(lambda x: 'CB' if '양배추' in x else x)
    
    return data
english(trade, '품목명')

,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,year,month
0,2019-01,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990,2019,1
1,2019-01,양파,821330,222,4003206,1118,-896,2019,1
2,2019-01,쪽파,60,1,93405,128,-127,2019,1
3,2019-01,BC,160,1,638913,563,-562,2019,1
4,2019-01,CB,0,0,7580,38,-38,2019,1
...,...,...,...,...,...,...,...,...,...
1269,2023-02,포포(papaw)[파파야(papaya)],0,0,23830,71,-71,2023,2
1270,2023-02,사과,135165,351,0,0,351,2023,2
1271,2023-02,배,2206012,5411,1,0,5411,2023,2
1272,2023-02,신 체리[프루너스 체라서스(Prunus cerasus)],5,0,0,0,0,2023,2


In [215]:
trade = trade.rename(columns={'품목명':'item'})

In [216]:
filtered = trade[(trade['item'] == 'TG') | (trade['item'] == 'BC') | (trade['item'] == 'RD') | (trade['item'] == 'CR') | (trade['item'] == 'CB')]
# filtered.drop('기간', axis=1, inplace=True)
filtered

,기간,item,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,year,month
3,2019-01,BC,160,1,638913,563,-562,2019,1
4,2019-01,CB,0,0,7580,38,-38,2019,1
5,2019-01,CB,184650,94,395802,90,4,2019,1
8,2019-01,CR,23150,22,7466150,2955,-2934,2019,1
12,2019-01,RD,2627,23,94529,464,-441,2019,1
...,...,...,...,...,...,...,...,...,...
1250,2023-02,CB,13188,13,377456,104,-91,2023,2
1253,2023-02,CR,22510,20,9260020,3758,-3737,2023,2
1254,2023-02,RD,4000,4,2,0,4,2023,2
1258,2023-02,RD,1319,14,104566,454,-440,2023,2


In [217]:
filtered = pd.get_dummies(filtered.drop('기간', axis=1))
filtered['기간'] = filtered['year'].astype(str) + '-' + filtered['month'].astype(str)
filtered.drop(['year', 'month'], axis=1, inplace=True)
filtered

,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,item_BC,item_CB,item_CR,item_RD,item_TG,기간
3,160,1,638913,563,-562,True,False,False,False,False,2019-1
4,0,0,7580,38,-38,False,True,False,False,False,2019-1
5,184650,94,395802,90,4,False,True,False,False,False,2019-1
8,23150,22,7466150,2955,-2934,False,False,True,False,False,2019-1
12,2627,23,94529,464,-441,False,False,False,True,False,2019-1
...,...,...,...,...,...,...,...,...,...,...,...
1250,13188,13,377456,104,-91,False,True,False,False,False,2023-2
1253,22510,20,9260020,3758,-3737,False,False,True,False,False,2023-2
1254,4000,4,2,0,4,False,False,False,True,False,2023-2
1258,1319,14,104566,454,-440,False,False,False,True,False,2023-2


In [218]:
train = train.drop(['timestamp', 'ID'], axis=1)
train = pd.get_dummies(train)
train['기간'] = train['year'].astype(str) + '-' + train['month'].astype(str)
# train.drop(['year', 'month','day'], axis=1, inplace=True)
train
X = train.drop(['supply(kg)', 'price(원/kg)'], axis=1)
y = train['price(원/kg)']

In [219]:
X

,year,month,day,item_BC,item_CB,item_CR,item_RD,item_TG,corporation_A,corporation_B,corporation_C,corporation_D,corporation_E,corporation_F,location_J,location_S,기간
0,2019,1,1,False,False,False,False,True,True,False,False,False,False,False,True,False,2019-1
1,2019,1,2,False,False,False,False,True,True,False,False,False,False,False,True,False,2019-1
2,2019,1,3,False,False,False,False,True,True,False,False,False,False,False,True,False,2019-1
3,2019,1,4,False,False,False,False,True,True,False,False,False,False,False,True,False,2019-1
4,2019,1,5,False,False,False,False,True,True,False,False,False,False,False,True,False,2019-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,2023,2,27,False,False,False,True,False,False,False,False,False,False,True,True,False,2023-2
59393,2023,2,28,False,False,False,True,False,False,False,False,False,False,True,True,False,2023-2
59394,2023,3,1,False,False,False,True,False,False,False,False,False,False,True,True,False,2023-3
59395,2023,3,2,False,False,False,True,False,False,False,False,False,False,True,True,False,2023-3


In [220]:
X = pd.merge(filtered, X, on='기간', how='inner')

# '기간' 열 삭제
X.drop('기간', axis=1, inplace=True)

# 월별로 데이터프레임 그룹화
grouped_df = X.groupby(['year', 'month'])

# 각 그룹을 2D 배열로 변환하여 리스트에 저장
result = [grouped_df.get_group(x).values for x in grouped_df.groups]

# 각 배열을 동일한 형태로 맞추기 위해 패딩 적용
max_length = max(len(x) for x in result)
result_padded = [np.pad(x, ((0, max_length - len(x)), (0, 0)), mode='constant', constant_values=0) for x in result]

# 리스트를 넘파이 배열로 변환
result = np.array(result_padded)

# 3D 배열의 형태 확인
print(result.shape)

(50, 8463, 26)


In [221]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [240]:
import tensorflow as tf

# 데이터 표준화
scaler = StandardScaler()
reshaped_result = result.reshape(-1, result.shape[-1])
result_sc = scaler.fit_transform(reshaped_result)
result_sc = result_sc.reshape(result.shape)

# 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(32, 3, activation='relu', input_shape=(result.shape[1], result.shape[2])),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

# 바디 모델의 출력 형태 가져오기
output_shape = model.output_shape

# 헤더 부분 정의
header_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=output_shape[1:]),  # 입력 형태를 바디 모델의 출력 형태와 일치시킴
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu')
])

# 바디 모델과 헤더 모델 연결
combined_model = tf.keras.Sequential([
    header_model,
    model
])


# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_15 (Conv1D)          (None, 8461, 32)          2528      
                                                                 
 max_pooling1d_15 (MaxPooli  (None, 4230, 32)          0         
 ng1D)                                                           
                                                                 
 flatten_15 (Flatten)        (None, 135360)            0         
                                                                 
 dense_36 (Dense)            (None, 64)                8663104   
                                                                 
 dense_37 (Dense)            (None, 1)                 65        
                                                                 
Total params: 8665697 (33.06 MB)
Trainable params: 8665697 (33.06 MB)
Non-trainable params: 0 (0.00 Byte)
_____________

In [241]:
# 모델 학습
from tensorflow.keras.callbacks import EarlyStopping

es_callback = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
history = model.fit(result_sc, y, validation_split=0.2, batch_size=64, epochs=300, callbacks=[es_callback])

Epoch 1/300
1/1 [==============================] - 1s 772ms/step - loss: 2926402.0000 - mae: 1405.4879 - val_loss: 3001705.0000 - val_mae: 1569.5527
Epoch 2/300
1/1 [==============================] - 0s 223ms/step - loss: 2342936.7500 - mae: 1303.0916 - val_loss: 2391480.5000 - val_mae: 1435.4685
Epoch 3/300
1/1 [==============================] - 0s 239ms/step - loss: 1859245.6250 - mae: 1200.5881 - val_loss: 1855139.0000 - val_mae: 1293.8456
Epoch 4/300
1/1 [==============================] - 0s 224ms/step - loss: 1463062.2500 - mae: 1092.1808 - val_loss: 1418276.7500 - val_mae: 1147.6434
Epoch 5/300
1/1 [==============================] - 0s 221ms/step - loss: 1177365.3750 - mae: 979.7262 - val_loss: 1099117.3750 - val_mae: 1001.0746
Epoch 6/300
1/1 [==============================] - 0s 225ms/step - loss: 1015512.3750 - mae: 870.7474 - val_loss: 904961.3750 - val_mae: 861.3605
Epoch 7/300
1/1 [==============================] - 0s 225ms/step - loss: 975298.6250 - mae: 801.3522 - val_los

In [243]:
test = pd.get_dummies(test)
# test 데이터에 학습된 scaler를 적용
test = pd.DataFrame(scaler.transform(test), columns=test.columns)  # 결과 데이터를 다시 데이터프레임으로 변환

test = test[result.columns]  # train 데이터와 동일한 특징을 가져야 함
test_sc = scaler.transform(test)

ValueError: Data must be 1-dimensional, got ndarray of shape (1092, 1, 15) instead

In [253]:
# test = np.array(test, dtype=np.float32)  # 데이터 타입을 float32로 변환
# test = test.reshape(test.shape[0], 1, test.shape[1])  # 데이터의 형태를 (None, 1, 15)로 변경
y_pred = model.predict(test.reshape(test.shape[0], test.shape[1], 1))


ValueError: cannot reshape array of size 16380 into shape (1092,1,1)

In [245]:
test.shape

(1092, 1, 15)

In [9]:
filtered.loc[filtered['month'] == 3]

,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,year,month,item_BC,item_CB,item_CR,item_RD,item_TG
51,100,0,417756,503,-503,2019,3,True,False,False,False,False
52,0,0,4520,22,-22,2019,3,False,True,False,False,False
53,854925,354,356514,94,260,2019,3,False,True,False,False,False
56,17960,13,13353720,4953,-4941,2019,3,False,False,True,False,False
59,220,2,17808,109,-107,2019,3,False,False,False,True,False
64,2061,10,4599,12,-2,2019,3,False,False,False,False,True
357,0,0,978995,1008,-1008,2020,3,True,False,False,False,False
358,0,0,13293,52,-52,2020,3,False,True,False,False,False
359,4472,6,3870094,1021,-1016,2020,3,False,True,False,False,False
362,16620,21,9053150,4228,-4206,2020,3,False,False,True,False,False


In [17]:
train.loc[(train['timestamp'] >= '2023-02') & (train['timestamp'] < '2023-03')]

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day
1492,TG_A_J_20230201,2023-02-01,TG,A,J,14483.2,2008.0,2023,2,1
1493,TG_A_J_20230202,2023-02-02,TG,A,J,16119.0,1978.0,2023,2,2
1494,TG_A_J_20230203,2023-02-03,TG,A,J,10930.0,1911.0,2023,2,3
1495,TG_A_J_20230204,2023-02-04,TG,A,J,30595.0,1864.0,2023,2,4
1496,TG_A_J_20230205,2023-02-05,TG,A,J,0.0,0.0,2023,2,5
...,...,...,...,...,...,...,...,...,...,...
59389,RD_F_J_20230224,2023-02-24,RD,F,J,466440.0,388.0,2023,2,24
59390,RD_F_J_20230225,2023-02-25,RD,F,J,250580.0,429.0,2023,2,25
59391,RD_F_J_20230226,2023-02-26,RD,F,J,0.0,0.0,2023,2,26
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,27


In [11]:
test

,ID,timestamp,item,corporation,location,year,month,day
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,4
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,5
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,6
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,7
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,8
...,...,...,...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,RD,F,J,2023,3,27
1088,RD_F_J_20230328,2023-03-28,RD,F,J,2023,3,28
1089,RD_F_J_20230329,2023-03-29,RD,F,J,2023,3,29
1090,RD_F_J_20230330,2023-03-30,RD,F,J,2023,3,30
